In [1]:
!pip install numpy==1.26.4 pandas==2.2.2 bottleneck==1.3.8 xarray zarr dask lightning matplotlib wandb cftime einops --quiet

import os
from datetime import datetime
import numpy as np
import xarray as xr
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import lightning.pytorch as pl

In [ ]:
pip uninstall torch torchvision torchaudio -y

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
data_path = "processed_data_cse151b_v2_corrupted_ssp245/processed_data_cse151b_v2_corrupted_ssp245.zarr"
data = xr.open_zarr(data_path)
data

<xarray.Dataset>
Dimensions:    (ssp: 4, time: 1021, latitude: 48, longitude: 72, x: 72, y: 48,
                member_id: 3)
Coordinates:
    lat        (x, y) float64 dask.array<chunksize=(72, 48), meta=np.ndarray>
  * latitude   (latitude) float64 -89.05 -85.26 -81.47 ... 81.47 85.26 89.05
    lon        (x, y) float64 dask.array<chunksize=(72, 48), meta=np.ndarray>
  * longitude  (longitude) float64 1.25 6.25 11.25 16.25 ... 346.2 351.2 356.2
  * member_id  (member_id) int64 0 1 2
  * ssp        (ssp) <U6 'ssp126' 'ssp245' 'ssp370' 'ssp585'
  * time       (time) object 2015-01-15 00:00:00 ... 2100-01-15 00:00:00
  * x          (x) float64 1.875 6.875 11.88 16.88 ... 341.9 346.9 351.9 356.9
  * y          (y) float64 -88.59 -84.82 -81.05 -77.28 ... 81.05 84.82 88.59
Data variables:
    BC         (ssp, time, latitude, longitude) float64 dask.array<chunksize=(1, 24, 48, 72), meta=np.ndarray>
    CH4        (ssp, time) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    CO2        (ssp, time) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    SO2        (ssp, time, latitude, longitude) float64 dask.array<chunksize=(1, 24, 48, 72), meta=np.ndarray>
    pr         (ssp, time, member_id, y, x) float32 dask.array<chunksize=(1, 24, 1, 48, 72), meta=np.ndarray>
    rsdt       (ssp, time, y, x) float32 dask.array<chunksize=(1, 24, 48, 72), meta=np.ndarray>
    tas        (ssp, time, member_id, y, x) float32 dask.array<chunksize=(1, 24, 1, 48, 72), meta=np.ndarray>
Attributes:
    original_member_ids:  ['r10i1p1f1', 'r11i1p1f1', 'r4i1p1f1']
    precipitation_units:  mm/day (converted from kg m-2 s-1)
    source:               CMIP6 data processed for CSE151B
    ssp:                  ssp126

In [3]:
# ----------------------------------------------
# 🔧 Imports & Setup
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import Trainer
from datetime import datetime
# ----------------------------------------------

# ----------------------------------------------
# 📦 Config (keep your original structure)
config = {
    "data": {
        "path": data_path,
        "input_vars": ["CO2", "SO2", "CH4", "BC", "rsdt"],
        "output_vars": ["tas", "pr"],
        "target_member_id": 0,
        "train_ssps": ["ssp126", "ssp370", "ssp585"],
        "test_ssp": "ssp245",
        "test_months": 360,
        "batch_size": 64,
        "num_workers": 0, # Previously 4
    },
    "model": {
        "type": "simple_cnn",
        "kernel_size": 3,
        "init_dim": 64,
        "depth": 4,
        "dropout_rate": 0.1,
    },
    "training": {
        "lr": 1e-3,
    },
    "trainer": {
        "max_epochs": 10,
        "accelerator": "gpu",
        "devices": "1",
        "precision": 32,
        "deterministic": True,
        "num_sanity_val_steps": 0,
    },
    "seed": 42,
}
pl.seed_everything(config["seed"])
# ----------------------------------------------

# ----------------------------------------------
# Helper: Latitude Weights
def get_lat_weights(lat_vals):
    lat_rad = np.deg2rad(lat_vals)
    weights = np.cos(lat_rad)
    return weights / np.mean(weights)

Seed set to 42


In [4]:
def get_lat_weights(latitude_values):
    lat_rad = np.deg2rad(latitude_values)
    weights = np.cos(lat_rad)
    return weights / np.mean(weights)

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride), nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.skip(identity)
        return self.relu(out)

class SimpleCNN(nn.Module):
    def __init__(self, n_input_channels, n_output_channels, kernel_size=3, init_dim=64, depth=4, dropout_rate=0.2):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(init_dim),
            nn.ReLU(inplace=True),
        )
        self.res_blocks = nn.ModuleList()
        current_dim = init_dim
        for i in range(depth):
            out_dim = current_dim * 2 if i < depth - 1 else current_dim
            self.res_blocks.append(ResidualBlock(current_dim, out_dim))
            if i < depth - 1:
                current_dim *= 2
        self.dropout = nn.Dropout2d(dropout_rate)
        self.final = nn.Sequential(
            nn.Conv2d(current_dim, current_dim // 2, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(current_dim // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(current_dim // 2, n_output_channels, kernel_size=1),
        )

    def forward(self, x):
        x = self.initial(x)
        for res_block in self.res_blocks:
            x = res_block(x)
        return self.final(self.dropout(x))


In [6]:
class Normalizer:
    def __init__(self):
        self.mean_in, self.std_in = None, None
        self.mean_out, self.std_out = None, None

    def set_input_statistics(self, mean, std):
        self.mean_in = mean
        self.std_in = std

    def set_output_statistics(self, mean, std):
        self.mean_out = mean
        self.std_out = std

    def normalize(self, data, data_type):
        if data_type == "input":
            return (data - self.mean_in) / self.std_in
        elif data_type == "output":
            return (data - self.mean_out) / self.std_out

    def inverse_transform_output(self, data):
        return data * self.std_out + self.mean_out


In [7]:
class ClimateDataset(Dataset):
    def __init__(self, inputs_dask, outputs_dask, output_is_normalized=True):
        self.size = inputs_dask.shape[0]
        print(f"Creating dataset with {self.size} samples...")

        inputs_np = inputs_dask.compute()
        outputs_np = outputs_dask.compute()

        self.inputs = torch.from_numpy(inputs_np).float()
        self.outputs = torch.from_numpy(outputs_np).float()

        if torch.isnan(self.inputs).any() or torch.isnan(self.outputs).any():
            raise ValueError("NaNs found in dataset")

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]


class ClimateDataModule(pl.LightningDataModule):
    def __init__(
        self,
        path,
        input_vars,
        output_vars,
        train_ssps,
        test_ssp,
        target_member_id,
        # val_split=0.1,
        test_months=360,
        batch_size=32,
        num_workers=0,
        seed=42,
    ):
        super().__init__()
        self.path = path
        self.input_vars = input_vars
        self.output_vars = output_vars
        self.train_ssps = train_ssps
        self.test_ssp = test_ssp
        self.target_member_id = target_member_id
        # self.val_split = val_split
        self.test_months = test_months
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        self.normalizer = Normalizer()

    def prepare_data(self):
        assert os.path.exists(self.path), f"Data path not found: {self.path}"

    def setup(self, stage=None):
        ds = xr.open_zarr(self.path, consolidated=False, chunks={"time": 24})
        spatial_template = ds["rsdt"].isel(time=0, ssp=0, drop=True)

        def load_ssp(ssp):
            input_dask, output_dask = [], []
            for var in self.input_vars:
                da_var = ds[var].sel(ssp=ssp)
                if "latitude" in da_var.dims:
                    da_var = da_var.rename({"latitude": "y", "longitude": "x"})
                if "member_id" in da_var.dims:
                    da_var = da_var.sel(member_id=self.target_member_id)
                if set(da_var.dims) == {"time"}:
                    da_var = da_var.broadcast_like(spatial_template).transpose("time", "y", "x")
                input_dask.append(da_var.data)

            for var in self.output_vars:
                da_out = ds[var].sel(ssp=ssp, member_id=self.target_member_id)
                if "latitude" in da_out.dims:
                    da_out = da_out.rename({"latitude": "y", "longitude": "x"})
                output_dask.append(da_out.data)

            return da.stack(input_dask, axis=1), da.stack(output_dask, axis=1)

        train_input, train_output, val_input, val_output = [], [], None, None

        for ssp in self.train_ssps:
            x, y = load_ssp(ssp)
            if ssp == "ssp370":
                val_input = x[-self.test_months:]
                val_output = y[-self.test_months:]
                train_input.append(x[:-self.test_months])
                train_output.append(y[:-self.test_months])
            else:
                train_input.append(x)
                train_output.append(y)

        train_input = da.concatenate(train_input, axis=0)
        train_output = da.concatenate(train_output, axis=0)

        self.normalizer.set_input_statistics(
            mean=da.nanmean(train_input, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_input, axis=(0, 2, 3), keepdims=True).compute(),
        )
        self.normalizer.set_output_statistics(
            mean=da.nanmean(train_output, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_output, axis=(0, 2, 3), keepdims=True).compute(),
        )

        train_input_norm = self.normalizer.normalize(train_input, "input")
        train_output_norm = self.normalizer.normalize(train_output, "output")
        val_input_norm = self.normalizer.normalize(val_input, "input")
        val_output_norm = self.normalizer.normalize(val_output, "output")

        test_input, test_output = load_ssp(self.test_ssp)
        test_input = test_input[-self.test_months:]
        test_output = test_output[-self.test_months:]
        test_input_norm = self.normalizer.normalize(test_input, "input")

        self.train_dataset = ClimateDataset(train_input_norm, train_output_norm)
        self.val_dataset = ClimateDataset(val_input_norm, val_output_norm)
        self.test_dataset = ClimateDataset(test_input_norm, test_output, output_is_normalized=False)

        self.lat = spatial_template.y.values
        self.lon = spatial_template.x.values
        self.area_weights = xr.DataArray(get_lat_weights(self.lat), dims=["y"], coords={"y": self.lat})

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,
                          num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def get_lat_weights(self):
        return self.area_weights

    def get_coords(self):
        return self.lat, self.lon

In [8]:
import pandas as pd

class ClimateEmulationModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4):
        super().__init__()
        self.model = model
        self.save_hyperparameters(ignore=['model']) # Save all hyperparameters except the model to self.hparams.<param_name>
        self.criterion = nn.MSELoss()
        self.normalizer = None
        self.val_preds, self.val_targets = [], []
        self.test_preds, self.test_targets = [], []

    def forward(self, x):
        return self.model(x)

    def on_fit_start(self):
        self.normalizer = self.trainer.datamodule.normalizer  # Get the normalizer from the datamodule (see above)

    def training_step(self, batch, batch_idx):
        x, y = batch # Unpack inputs and targets (this is the output of the _getitem_ method in the Dataset above)
        y_hat = self(x)   # Forward pass
        loss = self.criterion(y_hat, y)  # Calculate loss
        self.log("train/loss", loss)  # Log loss for tracking
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val/loss", loss)

        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = self.normalizer.inverse_transform_output(y.detach().cpu().numpy())
        self.val_preds.append(y_hat_np)
        self.val_targets.append(y_np)

        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions and ground truths from each val step/batch into one array
        preds = np.concatenate(self.val_preds, axis=0)
        trues = np.concatenate(self.val_targets, axis=0)
        self._evaluate(preds, trues, phase="val")
        np.save("val_preds.npy", preds)
        np.save("val_trues.npy", trues)
        self.val_preds.clear()
        self.val_targets.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = y.detach().cpu().numpy()
        self.test_preds.append(y_hat_np)
        self.test_targets.append(y_np)

    def on_test_epoch_end(self):
        # Concatenate all predictions and ground truths from each test step/batch into one array
        preds = np.concatenate(self.test_preds, axis=0)
        trues = np.concatenate(self.test_targets, axis=0)
        self._evaluate(preds, trues, phase="test")
        self._save_submission(preds)
        self.test_preds.clear()
        self.test_targets.clear()

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    def _evaluate(self, preds, trues, phase="val"):
        datamodule = self.trainer.datamodule
        area_weights = datamodule.get_lat_weights()
        lat, lon = datamodule.get_coords()
        time = np.arange(preds.shape[0])
        output_vars = datamodule.output_vars

        for i, var in enumerate(output_vars):
            p = preds[:, i]
            t = trues[:, i]
            p_xr = xr.DataArray(p, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
            t_xr = xr.DataArray(t, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

            # RMSE
            rmse = np.sqrt(((p_xr - t_xr) ** 2).weighted(area_weights).mean(("time", "y", "x")).item())
            # RMSE of time-mean
            mean_rmse = np.sqrt(((p_xr.mean("time") - t_xr.mean("time")) ** 2).weighted(area_weights).mean(("y", "x")).item())
            # MAE of time-stddev
            std_mae = np.abs(p_xr.std("time") - t_xr.std("time")).weighted(area_weights).mean(("y", "x")).item()

            print(f"[{phase.upper()}] {var}: RMSE={rmse:.4f}, Time-Mean RMSE={mean_rmse:.4f}, Time-Stddev MAE={std_mae:.4f}")
            self.log_dict({
                f"{phase}/{var}/rmse": rmse,
                f"{phase}/{var}/time_mean_rmse": mean_rmse,
                f"{phase}/{var}/time_std_mae": std_mae,
            })

    def _save_submission(self, predictions):
        datamodule = self.trainer.datamodule
        lat, lon = datamodule.get_coords()
        output_vars = datamodule.output_vars
        time = np.arange(predictions.shape[0])

        rows = []
        for t_idx, t in enumerate(time):
            for var_idx, var in enumerate(output_vars):
                for y_idx, y in enumerate(lat):
                    for x_idx, x in enumerate(lon):
                        row_id = f"t{t_idx:03d}_{var}_{y:.2f}_{x:.2f}"
                        pred = predictions[t_idx, var_idx, y_idx, x_idx]
                        rows.append({"ID": row_id, "Prediction": pred})

        df = pd.DataFrame(rows)
        os.makedirs("submissions", exist_ok=True)
        filepath = f"submissions/kaggle_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filepath, index=False)
        print(f"✅ Submission saved to: {filepath}")

In [9]:
# ----------------------------------------------
# 🧑‍💻 Run the Full Pipeline
# Load data
datamodule = ClimateDataModule(
    path=config["data"]["path"],
    input_vars=config["data"]["input_vars"],
    output_vars=config["data"]["output_vars"],
    train_ssps=config["data"]["train_ssps"],
    test_ssp=config["data"]["test_ssp"],
    target_member_id=config["data"]["target_member_id"],
    test_months=config["data"]["test_months"],
    batch_size=config["data"]["batch_size"],
    num_workers=config["data"]["num_workers"],
    seed=config["seed"]
)

datamodule.prepare_data()
datamodule.setup()

# Initialize model
model = SimpleCNN(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    kernel_size=config["model"]["kernel_size"],
    init_dim=config["model"]["init_dim"],
    depth=config["model"]["depth"],
    dropout_rate=config["model"]["dropout_rate"]
)

# Lightning wrapper
climate_model = ClimateEmulationModule(
    model=model,
    learning_rate=config["training"]["lr"]
)

Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


In [10]:
# Train
trainer = Trainer(**config["trainer"])
trainer.fit(climate_model, datamodule=datamodule)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | model     | SimpleCNN | 10.7 M | train
1 | criterion | MSELoss   | 0      | train
------------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.923    Total estimated model params size (MB)
47        Modules in train mode
0         Modules in eval mode
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if y

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=4.3921, Time-Mean RMSE=2.9801, Time-Stddev MAE=1.0870
[VAL] pr: RMSE=2.7542, Time-Mean RMSE=1.0023, Time-Stddev MAE=1.4704


Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=4.6929, Time-Mean RMSE=3.4156, Time-Stddev MAE=1.1408
[VAL] pr: RMSE=2.6764, Time-Mean RMSE=0.8260, Time-Stddev MAE=1.4429


`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
config["trainer"]["max_epochs"] = 20

In [16]:
# Save model checkpoint manually after training completes
trainer.save_checkpoint("climate_model_epoch10.ckpt")

In [17]:
# Recreate your model instance first (same args as before)
model = SimpleCNN(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    kernel_size=config["model"]["kernel_size"],
    init_dim=config["model"]["init_dim"],
    depth=config["model"]["depth"],
    dropout_rate=config["model"]["dropout_rate"]
)

# Load full LightningModule from checkpoint
climate_model = ClimateEmulationModule.load_from_checkpoint(
    checkpoint_path="climate_model_epoch10.ckpt",
    model=model,
    learning_rate=config["training"]["lr"]
)

# Update epochs if you want to continue further
# config["trainer"]["max_epochs"] = 

# Recreate trainer and continue training
trainer = Trainer(**config["trainer"])
trainer.fit(climate_model, datamodule=datamodule)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | model     | SimpleCNN | 10.7 M | train
1 | criterion | MSELoss   | 0      | train
------------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.923    Total estimated model params size (MB)
47        Modules in train mode
0         Modules in eval mode
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if y

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=2.5183, Time-Mean RMSE=1.2098, Time-Stddev MAE=0.6618
[VAL] pr: RMSE=2.1096, Time-Mean RMSE=0.4211, Time-Stddev MAE=0.9301


Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=2.6200, Time-Mean RMSE=1.5575, Time-Stddev MAE=0.7563
[VAL] pr: RMSE=2.0799, Time-Mean RMSE=0.4725, Time-Stddev MAE=0.9136


`Trainer.fit` stopped: `max_epochs=20` reached.


In [23]:
climate_model = ClimateEmulationModule.load_from_checkpoint(
    checkpoint_path="climate_model_epoch30.ckpt",
    model=model,
    learning_rate=config["training"]["lr"]
)
trainer.fit(climate_model, datamodule=datamodule)

Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\20187\CSE 151B\Project\lightning_logs\version_20\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | model     | SimpleCNN | 10.7 M | train
1 | criterion | MSELoss   | 0      | train
------------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.923    Total estimated model params size (MB)
47        Modules in train mode
0         Modules in eval mode
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\20187\anaco

In [24]:
# Validate + Test + CSV Generation
trainer.validate(climate_model, datamodule=datamodule)
trainer.test(climate_model, datamodule=datamodule)
# ----------------------------------------------

Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=2.6200, Time-Mean RMSE=1.5575, Time-Stddev MAE=0.7563
[VAL] pr: RMSE=2.0799, Time-Mean RMSE=0.4725, Time-Stddev MAE=0.9136


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val/loss          │    0.18110010027885437    │
│        val/pr/rmse        │    2.0799059867858887     │
│   val/pr/time_mean_rmse   │    0.4725455045700073     │
│    val/pr/time_std_mae    │    0.9135953187942505     │
│       val/tas/rmse        │    2.6200196743011475     │
│  val/tas/time_mean_rmse   │     1.557510256767273     │
│   val/tas/time_std_mae    │    0.7563069462776184     │
└───────────────────────────┴───────────────────────────┘

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

[TEST] tas: RMSE=290.4695, Time-Mean RMSE=290.4334, Time-Stddev MAE=3.3395
[TEST] pr: RMSE=4.1874, Time-Mean RMSE=3.8210, Time-Stddev MAE=1.2334
✅ Submission saved to: submissions/kaggle_submission_20250515_051931.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/pr/rmse        │     4.187388896942139     │
│  test/pr/time_mean_rmse   │    3.8209924697875977     │
│   test/pr/time_std_mae    │     1.233386754989624     │
│       test/tas/rmse       │     290.469482421875      │
│  test/tas/time_mean_rmse  │     290.4334411621094     │
│   test/tas/time_std_mae   │    3.3395166397094727     │
└───────────────────────────┴───────────────────────────┘

[{'test/tas/rmse': 290.469482421875,
  'test/tas/time_mean_rmse': 290.4334411621094,
  'test/tas/time_std_mae': 3.3395166397094727,
  'test/pr/rmse': 4.187388896942139,
  'test/pr/time_mean_rmse': 3.8209924697875977,
  'test/pr/time_std_mae': 1.233386754989624}]

## Save Best & Every 5 Epochs

In [14]:
from pytorch_lightning.callbacks import ModelCheckpoint

# Save best val_loss model
best_ckpt_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="best-{epoch:02d}-{val_loss:.4f}",
    save_top_k=1,
    monitor="val/loss",
    mode="min"
)

# Save every 5 epochs
periodic_ckpt_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="epoch{epoch:02d}",
    save_top_k=-1,              # Keep all periodic checkpoints
    every_n_epochs=5            # Save every 5 epochs
)

## Handling CNN Baseline Overfitting (Unreported In .ipynb)

In [15]:
config["model"]["dropout_rate"] = 0.3
config["data"]["batch_size"] = 32
config["training"]["lr"] = 8e-4
config["trainer"]["precision"] = 16

In [16]:
# Update epochs if you want to continue further
config["trainer"]["max_epochs"] = 30

In [17]:
# Recreate your model instance first (same args as before)
model = SimpleCNN(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    kernel_size=config["model"]["kernel_size"],
    init_dim=config["model"]["init_dim"],
    depth=config["model"]["depth"],
    dropout_rate=config["model"]["dropout_rate"]
)

# Load full LightningModule from checkpoint
climate_model = ClimateEmulationModule.load_from_checkpoint(
    checkpoint_path="checkpoints/epochepoch=29.ckpt",
    model=model,
    learning_rate=config["training"]["lr"]
)

# Trainer
trainer = Trainer(
    **config["trainer"],
    callbacks=[best_ckpt_callback, periodic_ckpt_callback]
)
trainer.fit(climate_model, datamodule=datamodule)

C:\Users\20187\anaconda3\Lib\site-packages\lightning_fabric\connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\20187\CSE 151B\Project\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | Mode 
------------------------------------------------
0 | model     | SimpleCNN | 10.7 M | train
1 | criterion | MSELoss   | 0      | train
------------------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.923    Total estimated model params size (MB)
47        Modules in train mode
0         Modules in eval mode
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
C:\Users\20187\anaconda3\Lib\site-packages\pyt

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=1.5018, Time-Mean RMSE=0.5906, Time-Stddev MAE=0.3692
[VAL] pr: RMSE=1.9586, Time-Mean RMSE=0.2543, Time-Stddev MAE=0.7600


Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=1.5676, Time-Mean RMSE=0.7390, Time-Stddev MAE=0.3603
[VAL] pr: RMSE=1.9519, Time-Mean RMSE=0.2478, Time-Stddev MAE=0.7582


`Trainer.fit` stopped: `max_epochs=30` reached.


In [19]:
# Validate + Test + CSV Generation
trainer.validate(climate_model, datamodule=datamodule)
trainer.test(climate_model, datamodule=datamodule)
# ----------------------------------------------

Creating dataset with 2703 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Creating dataset with 360 samples...


C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Validation: |                                                                                    | 0/? [00:00<…

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

[TEST] tas: RMSE=291.3141, Time-Mean RMSE=291.2725, Time-Stddev MAE=3.5250
[TEST] pr: RMSE=4.2762, Time-Mean RMSE=3.8072, Time-Stddev MAE=1.3873
✅ Submission saved to: submissions/kaggle_submission_20250515_171308.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/pr/rmse        │     4.276188850402832     │
│  test/pr/time_mean_rmse   │     3.807217836380005     │
│   test/pr/time_std_mae    │    1.3872519731521606     │
│       test/tas/rmse       │     291.3140869140625     │
│  test/tas/time_mean_rmse  │    291.27252197265625     │
│   test/tas/time_std_mae   │    3.5249547958374023     │
└───────────────────────────┴───────────────────────────┘

[{'test/tas/rmse': 291.3140869140625,
  'test/tas/time_mean_rmse': 291.27252197265625,
  'test/tas/time_std_mae': 3.5249547958374023,
  'test/pr/rmse': 4.276188850402832,
  'test/pr/time_mean_rmse': 3.807217836380005,
  'test/pr/time_std_mae': 1.3872519731521606}]

In [41]:
# Validate + Test + CSV Generation
trainer.validate(climate_model, datamodule=datamodule)
trainer.test(climate_model, datamodule=datamodule)
# ----------------------------------------------

Creating dataset with 2703 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Validation: |                                                                                    | 0/? [00:00<…

[VAL] tas: RMSE=1.6417, Time-Mean RMSE=0.8057, Time-Stddev MAE=0.3777
[VAL] pr: RMSE=1.9570, Time-Mean RMSE=0.2708, Time-Stddev MAE=0.7904


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val/loss          │    0.1557563692331314     │
│        val/pr/rmse        │    1.9569790363311768     │
│   val/pr/time_mean_rmse   │    0.2707831859588623     │
│    val/pr/time_std_mae    │    0.7903997898101807     │
│       val/tas/rmse        │    1.6416664123535156     │
│  val/tas/time_mean_rmse   │     0.805653989315033     │
│   val/tas/time_std_mae    │    0.3777344524860382     │
└───────────────────────────┴───────────────────────────┘

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\20187\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

[TEST] tas: RMSE=290.8429, Time-Mean RMSE=290.8008, Time-Stddev MAE=3.5328
[TEST] pr: RMSE=4.1636, Time-Mean RMSE=3.7179, Time-Stddev MAE=1.3353
✅ Submission saved to: submissions/kaggle_submission_20250515_065412.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/pr/rmse        │    4.1635942459106445     │
│  test/pr/time_mean_rmse   │     3.717919111251831     │
│   test/pr/time_std_mae    │     1.335269570350647     │
│       test/tas/rmse       │     290.8428955078125     │
│  test/tas/time_mean_rmse  │     290.8007507324219     │
│   test/tas/time_std_mae   │     3.532803535461426     │
└───────────────────────────┴───────────────────────────┘

[{'test/tas/rmse': 290.8428955078125,
  'test/tas/time_mean_rmse': 290.8007507324219,
  'test/tas/time_std_mae': 3.532803535461426,
  'test/pr/rmse': 4.1635942459106445,
  'test/pr/time_mean_rmse': 3.717919111251831,
  'test/pr/time_std_mae': 1.335269570350647}]